In [ ]:
"""
TITLE:           codev-01.ipynb
DESCRIPTION:     Set up a custom a Google Colab, Jupyter Notebook, cloud
                 environment that includes CPU/GPU recognition, customized GIT,
                 Google Drive connectivity, a "~/Projects" directory, Python
                 virtual environments, Python Libraries, SSH terminal access via
                 ngrok or Serveo, customized Vim, and more. The codev repository
                 offers more than is expressed here, so if needed, fork this
                 repository and customize to your requirements.
AUTHOR:          Christopher Smiga
DATE:            201125-1211
VERSION:         20.4.5-1
"""

# DEFINE ENVIRONMENT
! hostname codev-01
! git config --global user.name "MY_USER_NAME" \
    && git config --global user.email "MY_EMAIL_ADDRESS"

# UPDATE OPERATING SYSTEM
! DEBIAN_FRONTEND=noninteractive \
    && apt-get update --yes --fix-missing \
    && apt-get upgrade --yes --no-install-recommends \
    && apt-get install --yes --no-install-recommends -o=Dpkg::Use-Pty=0 \
        atop \
        chromium-chromedriver \
        cpp \
        cpustat \
        ctop \
        cython3 \
        default-jdk \
        default-jdk-doc \
        default-jre \
        dnsutils \
        ffmpeg \
        file \
        firefox-geckodriver \
        g++ \
        golang \
        htop \
        ifstat \
        iftop \
        itop \
        iputils-arping \
        iputils-clockdiff \
        iputils-ping \
        iputils-tracepath \
        libprotoc-dev \
        net-tools \
        openssh-server \
        protobuf-compiler \
        pwgen \
        sntop \
        vim \
        virtualenv \
    && apt-get autoremove \
    && apt-get purge

# MOUNT GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/gdrive')

# SET UP SYMBOLIC LINK TO GOOGLE DRIVE
! ln -s /content/gdrive/My\ Drive/Colab\ Notebooks ~/Colab\ Notebooks

# SET UP SSH KEY AND CONFIGURATION FILES FOR USER
! if [ ! -d "~/.ssh" ]; \
  then \
      echo "INFO: Directory ~/.ssh does not exist. Creating directory."; \
      echo "INFO: Setting up SSH keys and configuration files for user"; \
      mkdir ~/.ssh; \
      chmod 700 ~/.ssh; \
      touch ~/.ssh/known_hosts; \
      chmod 644 ~/.ssh/known_hosts; \
      cp /content/gdrive/My\ Drive/Colab\ Notebooks/os_configs/root/.ssh/authorized_keys ~/.ssh/authorized_keys; \
      cp /content/gdrive/My\ Drive/Colab\ Notebooks/os_configs/root/.ssh/config ~/.ssh/config; \
      chmod 600 ~/.ssh/authorized_keys ~/.ssh/config; \
      mkdir ~/.ssh/configs ~/.ssh/keys; \
      chmod 700 ~/.ssh/configs ~/.ssh/keys; \
      cp /content/gdrive/My\ Drive/Colab\ Notebooks/os_configs/root/.ssh/configs/* ~/.ssh/configs; \
      cp /content/gdrive/My\ Drive/Colab\ Notebooks/os_configs/root/.ssh/keys/* ~/.ssh/keys; \
      chmod 600 ~/.ssh/configs/* ~/.ssh/keys/*; \
      ln -s ~/.ssh/keys/id_rsa-MY_USER_NAME ~/.ssh/id_rsa; \
  else \
      echo "INFO: Directory ~/.ssh already exist"; \
  fi

# CONFIGURE SSHD
! echo "INFO: Updating /etc/ssh/sshd_config file" \
    && sed -i 's/#PermitRootLogin\ prohibit-password/PermitRootLogin\ yes/g' /etc/ssh/sshd_config \
    && sed -i 's/#PasswordAuthentication\ yes/PasswordAuthentication\ yes/g' /etc/ssh/sshd_config \
    && sed -i 's/#AuthorizedKeysFile\	.ssh\/authorized_keys\ .ssh\/authorized_keys2/AuthorizedKeysFile\ .ssh\/authorized_keys\ .ssh\/authorized_keys2/g' /etc/ssh/sshd_config \
    && sed -i 's/#AllowTcpForwarding\ yes/AllowTcpForwarding\ yes/g' /etc/ssh/sshd_config

# START SSHD
! service ssh start \
    && service ssh status

# DOWNLOAD TEMPORARY FILES
! echo "INFO: Downloading temporary files"
! git clone git@github.com:MY_USER_NAME/codev.git ~/tmp

# COPY SYSTEM FILES
! if [ ! -f "/etc/motd" ]; \
  then \
      echo "INFO: Copying system files"; \
      cp ~/tmp/os_configs/etc/motd /etc/motd; \
      chmod 644 /etc/motd; \
  else \
      echo "INFO: system files already exist"; \
  fi

# CREATE USER HOME PATH
! echo "INFO: Creating user HOME path" \
  && export HOME=~ \
  && echo "export HOME=~" >> ~/.bashrc

# COPY FILES FOR USER ENVIRONMENT
! if [ ! -f "~/.vimrc" ]; \
  then \
      echo "INFO: File ~/.vimrc does not exist. Copying vim files to user environment"; \
      cp ~/tmp/os_configs/root/.vimrc ~/.vimrc; \
      cp -R ~/tmp/os_configs/root/.vim ~/.vim; \
      chmod 600 ~/.vimrc; \
      chmod 700 ~/.vim; \
  else \
      echo "INFO: File ~/.vimrc already exist"; \
  fi

# CREATE PROJECTS DIRECTORY
! if [ -d "~/Projects" ]; \
  then \
      echo "INFO: Directory ~/Projects already exist"; \
  else \
      echo "INFO: Directory ~/Projects does not exist. Creating directory."; \
      mkdir ~/Projects; \
  fi

# SETUP PYTHON VIRTUAL ENVIRONMENTS
! virtualenv --python=/usr/bin/python3 ~/Projects/venv3 \
    && cd ~/Projects/venv3 \
    && . ./bin/activate \
    && cp ~/tmp/os_configs/root/Projects/requirements.txt ~/Projects \
    && pip install -r ~/Projects/requirements.txt \
    && deactivate

# GIT REPOSITORIES
! git clone git@github.com:MY_USER_NAME/cloud-config.git ~/Projects/cloud-config \
    && git clone git@github.com:MY_USER_NAME/codev.git ~/Projects/codev \
    && git clone git@github.com:MY_USER_NAME/funct.git ~/Projects/funct \
    && git clone git@github.com:MY_USER_NAME/nfunct.git ~/Projects/nfunct \
    && git clone git@github.com:MY_USER_NAME/funct_behaviors.git ~/Projects/funct_behaviors\
    && git clone git@github.com:MY_USER_NAME/nfunct_techniques.git ~/Projects/nfunct_techniques \
    && git clone git@github.com:MY_USER_NAME/nfunct_docker.git ~/Projects/nfunct_docker \
    && git clone git@github.com:MY_USER_NAME/nfunct_postman.git ~/Projects/nfunct_postman \
    && git clone git@gitlab.com:MY_USER_NAME/snippets.git ~/Projects/snippets

# REMOVE TEMPORARY FILES
! echo "INFO: Removing temporary files" \
    && rm -rf ~/tmp

# CPU OR GPU COMPUTATION IS SET
! if [ -e "/dev/nvidia0" ] && [ -e "/dev/nvidiactl" ]; \
  then \
      echo "INFO: GPU computation is set"; \
      export LD_LIBRARY_PATH="/usr/lib/x86_64-linux-gnu"; \
      export LD_PRELOAD="/usr/lib/x86_64-linux-gnu/libnvidia-ml.so"; \
      export ONNX_ML=1; \
      echo "" >> ~/.bashrc; \
      echo "ONNX_ML=1" >> ~/.bashrc; \
      echo "export LD_LIBRARY_PATH=/usr/lib/x86_64-linux-gnu" >> ~/.bashrc; \
      echo "export LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libnvidia-ml.so" >> ~/.bashrc; \
      nvidia-smi; \
      /usr/local/cuda/bin/nvcc --version; \
  else \
      echo "INFO: CPU computation is set"; \
      export ONNX_ML=1; \
      echo "" >> ~/.bashrc; \
      echo "export ONNX_ML=1" >> ~/.bashrc; \
  fi


In [ ]:
# NGROK: SSH TERMINAL VIA CLIENT SOFTWARE
# https://ngrok.com/docs

# PYTHON LIBRARIES
import getpass, json, random, string, time, urllib.request

# NGROK: GENERATE ROOT USER PASSWORD
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))
! echo root:$password | chpasswd

# NGROK: DOWNLOAD AND INSTALL
! if [ ! -f "/opt/ngrok/ngrok" ]; \
  then \
      echo "INFO: File ngrok does not exist. Downloading and installing ngrok."; \
      mkdir /opt/ngrok; \
      wget --continue --no-clobber --directory-prefix=/opt/ngrok https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip; \
      unzip -n /opt/ngrok/ngrok-stable-linux-amd64.zip -d /opt/ngrok; \
      rm -f /opt/ngrok/ngrok-stable-linux-amd64.zip; \
      chmod 755 /opt/ngrok/ngrok; \
  else \
     echo "INFO: File ngrok already exist"; \
  fi

# NGROK: OBTAIN AUTHORIZATION TOKEN FROM URL. 
print("Copy authtoken from https://dashboard.ngrok.com/auth")
ngrokAuthToken = getpass.getpass()
#ngrokAuthToken = '<NGROK_AUTHORIZATION_TOKEN>'

# NGROK: START PROXY AND WAIT FOR PROCESS TO COME UP
get_ipython().system_raw('/opt/ngrok/ngrok authtoken $ngrokAuthToken && /opt/ngrok/ngrok tcp 22 &')
print('Waiting for ngrok process to come up...')
time.sleep(10)

# NGROK: GET PUBLIC ADDRESS AND PRINT CONNECT COMMAND
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
    data = json.loads(response.read().decode())
    (host, port) = data['tunnels'][0]['public_url'][6:].split(':')

# NGROK: PRINT ROOT USER PASSWORD AND SSH 
print(f'Root password: {password}')
print(f'SSH command:   ssh -p {port} -o "StrictHostKeyChecking no" -l root 0.tcp.ngrok.io \n')


In [ ]:
# NGROK: SSH TERMINAL VIA REVERSE TUNNELING
# https://ngrok.com/docs#ssh-gateway

# PYTHON LIBRARIES
import getpass, random, string

password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

! echo root:$password | chpasswd
print(f'Root password: {password}')
print('SSH command:   ssh -p <port> -o "StrictHostKeyChecking no" -l root 0.tcp.ngrok.io \n')

! ssh -o "StrictHostKeyChecking no" -R 0:localhost:22 tunnel.us.ngrok.com tcp 22


In [ ]:
# SERVEO: SSH TERMINAL VIA REVERSE TUNNELING
# https://serveo.net/#manual

# PYTHON LIBRARIES
import getpass, random, string

password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))
alias = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(8))

! echo root:$password | chpasswd
print(f'Host alias:    {alias}')
print(f'Root password: {password}')
print(f'SSH command:   ssh -o "StrictHostKeyChecking no" -J serveo.net -l root {alias} \n')

! ssh -o "StrictHostKeyChecking no" -R $alias:22:localhost:22 serveo.net


In [ ]:
# LOCALHOST.RUN: SSH TERMINAL VIA REVERSE TUNNELING
# http://localhost.run


In [ ]:
# PACKETRIOT: SSH TERMINAL VIA REVERSE TUNNELING
# https://packetriot.com
